In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import seaborn as sns
import datetime
from group_lasso import GroupLasso
from sklearn.utils import resample, check_random_state

In [39]:
def standardize(X,y):
    # Standardize X to have mean = 0 std = 1
    # Standardize y to have mean = 0
    X_scaled = (X-np.mean(X,axis=0))/np.std(X,axis=0)
    y_scaled = y-np.mean(y)
    return X_scaled, y_scaled


def plot_coefficients(beta,alpha,alpha_opt=10,name=None):
    # Plotting regression coefficients vs lambda
    beta_opt = beta[:,np.argmin(np.abs(alpha-alpha_opt))]
    plt.figure()
    plt.plot(np.log10(alpha),beta.T,'-')
    plt.plot(np.log10(alpha_opt)*np.array([1,1]), [np.min(beta), np.max(beta)], 'k--')
    plt.xlabel(r'$\lambda$')
    plt.ylabel(r'$\beta$')
    plt.title(name)
    plt.show()

def plot_CV_MSE(alpha_vals, mse, alpha_opt, name=None):
    mse_mean = np.mean(mse,axis=1)
    mse_std = np.std(mse,axis=1)
    plt.figure()
    plt.errorbar(np.log10(alpha_vals), mse_mean, mse_std)
    plt.plot(np.log10([alpha_opt,alpha_opt]), [0,np.max(mse)],'k--')
    plt.xlabel(r'log($\lambda$)')
    plt.ylabel('MSE')
    plt.title(name)
    plt.show()
    print('Optimal value of lambda is: ', np.round(alpha_opt,3))
        

In [ ]:
df = pd.read_csv('energydata_complete.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

for i in range(len(df.columns)):
    plt.figure()
    plt.plot(df[df.columns[i]])
    plt.ylabel(df.columns[i])
    plt.show()

### Taking an n-hour mean

In [41]:
df = df.resample('24h').mean()

### Generating extra features to describe time
weekday: number [0,6]\
weekstatus: binary describing weekend (1) or not (0)\
NSM: Number of Seconds from Midnight

These are used for filtering the data

In [42]:
weekday = np.zeros(len(df))
weekstatus = np.zeros(len(df))
NSM = np.zeros(len(df))
month = np.zeros(len(df))

for i in range(len(df)):
    weekday[i] = df.index[i].weekday()
    weekstatus[i] = True if weekday[i] >= 5 else False  # False for workday, True for weekend
    NSM[i] = (df.index[i] - df.index[i].replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    month[i] = df.index[i].month

df['weekday'] = weekday
df['week status'] = weekstatus
df['NSM'] = NSM
df['month'] = month

In [ ]:
plt.figure()
plt.scatter(df['NSM'],df['Appliances'])
plt.xlabel('NSM')
plt.ylabel('Appliances')
plt.show()

### Filtering data and making training set
Example: Only february, after 16:00 and workday

In [44]:
#df_train = df[(df.index.month==2) & (df['NSM']>=16*3600) & (df['NSM']<24*3600)]
df_train = df[(df.index.month==2)]
df_train = df_train.drop(['weekday', 'week status','month','NSM'], axis=1) # dropping the features used for filtering

# Training data
y = np.array(df_train['Appliances']).reshape(-1,1)
X = np.array(df_train[df_train.columns[1:]])
X, y = standardize(X,y)

In [ ]:
plt.figure()
plt.scatter(X[:,19],y)
plt.show()

### Cross validation

In [46]:
# Creating array of penalties

n_alpha = 100 # Number of penalties
min_alpha = .01 # min penalty
max_alpha = 100 # Maximum penalty
alpha_vals = np.logspace(np.log10(min_alpha),np.log10(max_alpha),n_alpha)
alpha_vals = alpha_vals[::-1] # reversing array (some sklearn standard?)

#### CV Lasso

In [ ]:
cv_lasso = linear_model.LassoCV(cv=10, random_state=0, fit_intercept=False, alphas=alpha_vals).fit(X, y)

#### CV Ridge

In [ ]:
cv_ridge = linear_model.ElasticNetCV(cv=10, random_state=0, l1_ratio=0, fit_intercept=False, alphas=alpha_vals).fit(X, y)

#### CV Elastic net (0.5 ratio)

In [ ]:
cv_elnet = linear_model.ElasticNetCV(cv=10, random_state=0, l1_ratio=0.5, fit_intercept=False, alphas=alpha_vals).fit(X, y)

In [ ]:
print('Lasso score: ', cv_lasso.score(X,y))
print('Ridge score: ', cv_ridge.score(X,y))
print('Elastic net score: ', cv_elnet.score(X,y))

In [ ]:
plot_CV_MSE(alpha_vals,cv_lasso.mse_path_,cv_lasso.alpha_, 'Lasso')
plot_CV_MSE(alpha_vals,cv_ridge.mse_path_,cv_ridge.alpha_, 'Ridge')
plot_CV_MSE(alpha_vals,cv_elnet.mse_path_,cv_elnet.alpha_, 'Elastic net')

### Lasso

In [ ]:
beta_lasso = np.zeros((X.shape[1],n_alpha))

for i in range(n_alpha):
    reg = linear_model.Lasso(alpha=alpha_vals[i], max_iter = 1000, fit_intercept = False)
    reg.fit(X,y)
    beta_lasso[:,i] = reg.coef_

### Ridge

In [ ]:
beta_ridge = np.zeros((X.shape[1],n_alpha))

for i in range(n_alpha):
    reg = linear_model.ElasticNet(alpha=alpha_vals[i], max_iter = 1000, l1_ratio=0, fit_intercept = False)
    reg.fit(X,y)
    beta_ridge[:,i] = reg.coef_

### Elastic net

In [ ]:
beta_elnet = np.zeros((X.shape[1],n_alpha))

for i in range(n_alpha):
    reg = linear_model.ElasticNet(alpha=alpha_vals[i], max_iter = 1000, l1_ratio=0.5, fit_intercept = False)
    reg.fit(X,y)
    beta_elnet[:,i] = reg.coef_

### Grouped Lasso
Does not work yet

In [55]:
beta_glasso = np.zeros((X.shape[1],n_alpha))

for i in range(n_alpha):
    reg = GroupLasso(
    group_reg=5,
    l1_reg=1,
    frobenius_lipschitz=True,
    scale_reg="inverse_group_size",
    subsampling_scheme=1,
    supress_warning=True,
    n_iter=1000,
    tol=1e-3,
    )
    reg.fit(X, y)
    beta_glasso[:,i] = reg.coef_.reshape(-1,)

### Plotting coefficients vs lambda

In [ ]:
plot_coefficients(beta_ridge, alpha_vals, cv_ridge.alpha_, name='Ridge')
plot_coefficients(beta_lasso, alpha_vals, cv_lasso.alpha_, name='Lasso')
plot_coefficients(beta_elnet, alpha_vals, cv_elnet.alpha_, name='Elastic net')
#plot_coefficients(beta_glasso,alpha_vals,1,name='Grouped lasso')

In [ ]:
data = {'Feature': list(df_train.columns[1:])}
df_results = pd.DataFrame(data)
df_results['Lasso'] = beta_lasso[:,alpha_vals==cv_lasso.alpha_]
df_results['Ridge'] = beta_ridge[:,alpha_vals==cv_ridge.alpha_]
df_results['Elastic net'] = beta_elnet[:,alpha_vals==cv_elnet.alpha_]
df_results

### Testing with another month

In [ ]:
#df_test = df[(df.index.month==2) & (df['NSM']>=16*3600) & (df['NSM']<24*3600)]
df_test = df[(df.index.month==3)]
df_test = df_test.drop(['weekday', 'week status','month','NSM'], axis=1) # dropping the features used for filtering

# Testing data
y_t = np.array(df_test['Appliances']).reshape(-1,1)
X_t = np.array(df_test[df_test.columns[1:]])
X_t, y_t = standardize(X_t,y_t)

reg = linear_model.ElasticNet(alpha=cv_elnet.alpha_, max_iter = 1000, l1_ratio=0.5, fit_intercept = False)
reg = linear_model.Lasso(alpha=cv_lasso.alpha_, max_iter = 1000, fit_intercept = False)
reg.fit(X,y)
y_p = reg.predict(X_t)

plt.figure()
plt.scatter(y_p,y_t)
plt.show()
print(reg.score(X,y))
print(reg.score(X_t,y_t))